## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config3 import g_key
from config3 import weather_api_key

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Vardo,NO,70.3705,31.1107,46.85,71,100,9.40,overcast clouds,0.00,0
1,1,Port-Gentil,GA,-0.7193,8.7815,76.51,86,100,14.92,light rain,0.59,0
2,2,Clyde River,CA,70.4692,-68.5914,39.33,93,100,8.25,overcast clouds,0.00,0
3,3,Muros,ES,42.7762,-9.0603,66.24,73,6,15.46,clear sky,0.00,0
4,4,Bethel,US,41.3712,-73.4140,78.08,72,1,4.61,clear sky,0.00,0


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no) ")
snow_bool = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 66
What is the maximum temperature you would like for your trip? 88
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [47]:
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Vardo,NO,70.3705,31.1107,46.85,71,100,9.40,overcast clouds,0.00,0
1,1,Port-Gentil,GA,-0.7193,8.7815,76.51,86,100,14.92,light rain,0.59,0
2,2,Clyde River,CA,70.4692,-68.5914,39.33,93,100,8.25,overcast clouds,0.00,0
3,3,Muros,ES,42.7762,-9.0603,66.24,73,6,15.46,clear sky,0.00,0
4,4,Bethel,US,41.3712,-73.4140,78.08,72,1,4.61,clear sky,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
527,527,Shunyi,CN,40.1250,116.6453,73.35,78,0,8.95,light rain,0.13,0
528,528,Yichun,CN,47.7000,128.9000,66.65,67,3,1.92,clear sky,0.00,0
529,529,Sagua La Grande,CU,22.8067,-80.0756,86.99,74,84,11.70,broken clouds,0.00,0
530,530,Athabasca,CA,54.7169,-113.2854,69.62,40,88,19.93,overcast clouds,0.00,0


In [48]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
if rain_bool == "no" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif rain_bool == "no" and snow_bool == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_bool == "yes" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]

In [49]:
# 4a. Determine if there are any empty rows
clean_df = filtered_cities_df.dropna()

In [50]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
3,3,Muros,ES,42.7762,-9.0603,66.24,73,6,15.46,clear sky,0.0,0
4,4,Bethel,US,41.3712,-73.4140,78.08,72,1,4.61,clear sky,0.0,0
6,6,Port Hedland,AU,-20.3167,118.5667,68.05,28,0,10.36,clear sky,0.0,0
11,11,Atuona,PF,-9.8000,-139.0333,77.77,78,24,16.49,few clouds,0.0,0
13,13,Sao Filipe,CV,14.8961,-24.4956,78.57,88,47,1.83,scattered clouds,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
523,523,Lewistown,US,40.5992,-77.5714,77.79,91,90,0.00,overcast clouds,0.0,0
528,528,Yichun,CN,47.7000,128.9000,66.65,67,3,1.92,clear sky,0.0,0
529,529,Sagua La Grande,CU,22.8067,-80.0756,86.99,74,84,11.70,broken clouds,0.0,0
530,530,Athabasca,CA,54.7169,-113.2854,69.62,40,88,19.93,overcast clouds,0.0,0


In [51]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Muros,ES,66.24,clear sky,42.7762,-9.0603,
4,Bethel,US,78.08,clear sky,41.3712,-73.4140,
6,Port Hedland,AU,68.05,clear sky,-20.3167,118.5667,
11,Atuona,PF,77.77,few clouds,-9.8000,-139.0333,
13,Sao Filipe,CV,78.57,scattered clouds,14.8961,-24.4956,
14,Kapaa,US,86.32,clear sky,22.0752,-159.3190,
15,Bhadrachalam,IN,75.34,overcast clouds,17.6667,80.8833,
17,Broome,US,76.69,overcast clouds,42.2506,-75.8330,
21,Kieta,PG,82.31,overcast clouds,-6.2167,155.6333,
22,Espinosa,BR,77.22,broken clouds,-14.9261,-42.8192,


In [52]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [54]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [55]:

gmaps.configure(api_key= g_key)

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))